## Julia on Colaboratory ##

[Colaboratory](https://colab.research.google.com) does not provide native support for the [Julia programming language](https://julialang.org). However, since Colaboratory gives you root access to the machine that runs your notebook (the *“runtime”* in Colaboratory terminology), we can install Julia support by uploading a specially crafted Julia notebook  – *this* notebook. We then install Julia and [IJulia](https://github.com/JuliaLang/IJulia.jl) ([Jupyter](https://jupyter.org)/Colaboratory notebook support) and reload the notebook so that Colaboratory detects and initiates what we installed.

In brief:

1. **Run the cell below**
2. **Reload the page**
3. **Edit the notebook name and start hacking Julia code below**

**If your runtime resets**, either manually or if left idle for some time, **repeat steps 1 and 2**.

### Acknowledgements ###

This hack by Pontus Stenetorp is an adaptation of [James Bradbury’s original Colaboratory Julia hack](https://discourse.julialang.org/t/julia-on-google-colab-free-gpu-accelerated-shareable-notebooks/15319/27), that broke some time in September 2019 as Colaboratory increased their level of notebook runtime isolation. There also appears to be CUDA compilation support installed by default for each notebook runtime type in October 2019, which shaves off a good 15 minutes or so from the original hack’s installation time.

In [0]:
# Installation cell
%%shell
if ! command -v julia 2>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add Plots; add PyPlot; add IJulia; add Knet;"'
julia -e 'using Pkg; pkg"build Knet;"'

--2020-05-18 11:49:45--  https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz
Resolving julialang-s3.julialang.org (julialang-s3.julialang.org)... 151.101.2.49, 151.101.66.49, 151.101.130.49, ...
Connecting to julialang-s3.julialang.org (julialang-s3.julialang.org)|151.101.2.49|:443... connected.
HTTP request sent, awaiting response... 302 gce internal redirect trigger
Location: https://storage.googleapis.com/julialang2/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz [following]
--2020-05-18 11:49:45--  https://storage.googleapis.com/julialang2/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95929584 (91M) [application/x-gzip]
Saving to: ‘/tmp/julia.tar.gz’

/tmp/julia.tar.gz   100%[===================>]

In [1]:
using Knet
# Test if Knet is using gpu
Knet.gpu()

┌ Info: Precompiling Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1273


0

In [2]:
a = KnetArray(randn(4,4))
sigm.(a)

4×4 KnetArray{Float64,2}:
 0.554472  0.52774   0.497022  0.154425
 0.66232   0.292151  0.38498   0.668314
 0.841404  0.547291  0.380908  0.670597
 0.187157  0.518183  0.434034  0.78085 

In [3]:
#constant variables
random_seed         = 12345;
BATCH_SIZE          = 100;
LEARNING_RATE       = 0.15;
EPOCH               = 100;
SCALE               = 0.01;
ninputs             = 784;
noutputs            = 10;
atype = gpu() >= 0  ? KnetArray{Float32} : Array{Float32}

KnetArray{Float32,N} where N

In [5]:
using Pkg; for p in ("Knet","IterTools","Plots"); haskey(Pkg.installed(),p) || Pkg.add(p); end
using Knet, Printf, Random
include(Knet.dir("data", "mnist.jl"))

xtrn, ytrn, xtst, ytst = mnist()
dtrn = minibatch(xtrn[:,:,:,1:100], ytrn[1:100], BATCH_SIZE; xtype=atype)
dtst = minibatch(xtst[:,:,:,1:100], ytst[1:100], BATCH_SIZE; xtype=atype)

println.(summary.((xtrn, xtst, ytrn, ytst)));
println.(summary.((dtrn, dtst)));

┌ Info: Loading MNIST...
└ @ Main /root/.julia/packages/Knet/bTNMd/data/mnist.jl:33


28×28×1×60000 Array{Float32,4}
28×28×1×10000 Array{Float32,4}
60000-element Array{UInt8,1}
10000-element Array{UInt8,1}
1-element Knet.Data{Tuple{KnetArray{Float32,4},Array{UInt8,1}}}
1-element Knet.Data{Tuple{KnetArray{Float32,4},Array{UInt8,1}}}


In [6]:
# define onehot function
function to_onehot(x)
    onehot = zeros(10, 1)
    onehot[x, 1] = 1.0
    return onehot
end

to_onehot (generic function with 1 method)

In [7]:
## Loading the data and set random.seed 
Random.seed!(random_seed)
xtrn = reshape(xtrn, 784, 60000)
xtst = reshape(xtst, 784, 10000)
ytrn = hcat(map(to_onehot, ytrn)...)
ytst = hcat(map(to_onehot, ytst)...)
size(xtrn),size(xtst),size(ytrn),size(ytst)

((784, 60000), (784, 10000), (10, 60000), (10, 10000))

In [0]:
struct Linear; w; b; end

In [9]:
function Linear(xsize::Int, ysize::Int, SCALE=0.1)
    return Linear(SCALE * randn(ysize,xsize), zeros(ysize,1))
end

Linear

In [12]:
function custom_minibatch(X, Y)
    data = Any[]
    for i=1:BATCH_SIZE:size(X,2)
        j= min(i + BATCH_SIZE - 1, size(X,2))
        push!(data, ((X[:,i:j]), Y[:,i:j]))
    end
    return data
end

custom_minibatch (generic function with 1 method)

In [13]:
# define struct in W and b values 
model    = Linear(ninputs, noutputs)
trn_data = custom_minibatch(xtrn, ytrn)
size(trn_data), size(model.w), size(model.b)

((600,), (10, 784), (10, 1))

In [14]:
function softmax_function(W, b, x)
    calculator = exp.((W * x) .+ b)
    return calculator ./ sum(calculator, dims=1)
end

softmax_function (generic function with 1 method)

In [15]:
# define loss function
function softmax_back_and_loss(W, b, x, ygold)   
    loss = -sum(ygold .* (log.(softmax_function(W,b,x)))) / size(x,2)
    grad_b = sum( -(ygold-softmax_function(W,b,x)) / size(x,2), dims=2)
    grad_W =  -(ygold-softmax_function(W,b,x)) / size(x,2) * x'
    return loss, grad_W, grad_b
end

softmax_back_and_loss (generic function with 1 method)

In [16]:
# define train function
function train(W, b, data)
    totalcost = 0.0; numins = 0;
    for (x, y) in data
        loss, grad_W, grad_b =softmax_back_and_loss(W, b, x, y)        
        W  .-=  LEARNING_RATE * grad_W
        b  .-=  LEARNING_RATE * grad_b
        totalcost += loss .* size(x,2)
        numins += size(x,2)
    end
    avgcost = totalcost / numins
end

train (generic function with 1 method)

In [17]:
# define accuary function
function accuracy(ygold, ypred)
    correct = 0.0
    for i=1:size(ygold, 2)
        correct += findmax(ygold[:,i]; dims=1)[2] == findmax(ypred[:, i]; dims=1)[2] ? 1.0 : 0.0
    end
    return correct / size(ygold, 2)
end

accuracy (generic function with 1 method)

In [18]:
for i=1:EPOCH
    cost = train(model.w, model.b, trn_data)
    pred = softmax_function(model.w, model.b, xtrn)
    trnacc = accuracy(ytrn, pred)
    pred = softmax_function(model.w, model.b, xtst)
    tstacc = accuracy(ytst, pred)
    @printf("epoch: %d softloss: %g trn accuracy: %g tst accuracy: %g\n", i, cost, trnacc, tstacc)
end

epoch: 1 softloss: 0.501781 trn accuracy: 0.895183 tst accuracy: 0.9015
epoch: 2 softloss: 0.344278 trn accuracy: 0.905517 tst accuracy: 0.91
epoch: 3 softloss: 0.319777 trn accuracy: 0.910433 tst accuracy: 0.9129
epoch: 4 softloss: 0.306857 trn accuracy: 0.91345 tst accuracy: 0.9141
epoch: 5 softloss: 0.298475 trn accuracy: 0.9154 tst accuracy: 0.9163
epoch: 6 softloss: 0.292434 trn accuracy: 0.917267 tst accuracy: 0.9172
epoch: 7 softloss: 0.28779 trn accuracy: 0.9185 tst accuracy: 0.9182
epoch: 8 softloss: 0.284058 trn accuracy: 0.9197 tst accuracy: 0.919
epoch: 9 softloss: 0.280963 trn accuracy: 0.9203 tst accuracy: 0.919
epoch: 10 softloss: 0.278336 trn accuracy: 0.920983 tst accuracy: 0.9193
epoch: 11 softloss: 0.276062 trn accuracy: 0.921767 tst accuracy: 0.92
epoch: 12 softloss: 0.274067 trn accuracy: 0.922333 tst accuracy: 0.9201
epoch: 13 softloss: 0.272293 trn accuracy: 0.9229 tst accuracy: 0.9208
epoch: 14 softloss: 0.2707 trn accuracy: 0.923533 tst accuracy: 0.921
epoch: 1